In [1]:
# ======================================================
# 🔁 ENTRENAMIENTO CON VALIDACIÓN CRUZADA (K-FOLD)
# ======================================================
import torch
from torch import nn, optim, amp
from torchvision import models, datasets, transforms
from torchvision.transforms import InterpolationMode
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from torch.utils.data import Subset, DataLoader
import os, random

# ===== Configuración general =====
EPOCHS   = 20
LR       = 1e-3
PATIENCE = 7
KFOLDS   = 5
SEED     = 42
BATCH_SIZE = 32
ROOT     = "datasets"   # carpeta principal con 'con_pez/' y 'vacio/'
SAVE_DIR = "runs_kfold"  # carpeta donde se guardarán los modelos

os.makedirs(SAVE_DIR, exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Entrenando en dispositivo: {DEVICE.upper()}")

# ======================================================
# ⚙️ Transformaciones del dataset
# ======================================================
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(
        size=224, scale=(0.90, 1.00), ratio=(0.95, 1.05),
        interpolation=InterpolationMode.BICUBIC
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.RandomRotation(degrees=8, interpolation=InterpolationMode.BICUBIC)
    ], p=0.4),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.10, hue=0.02)
    ], p=0.5),
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.8))
    ], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ======================================================
# 📦 Dataset base
# ======================================================
full_ds = datasets.ImageFolder(ROOT, transform=train_tf)
targets = [y for _, y in full_ds.samples]  # etiquetas numéricas

# ======================================================
# ⚙️ K-Fold dividido estratificadamente
# ======================================================
kfold = StratifiedKFold(n_splits=KFOLDS, shuffle=True, random_state=SEED)

fold_metrics = {"accuracy": [], "precision": [], "recall": [], "f1": []}

best_global_acc = 0.0   # rastrea el mejor modelo global
best_global_state = None

# ======================================================
# 🔄 Bucle de entrenamiento K-Fold
# ======================================================
for fold, (train_idx, val_idx) in enumerate(kfold.split(np.zeros(len(targets)), targets), 1):
    print(f"\n📘 === FOLD {fold}/{KFOLDS} ===")

    # Subconjuntos
    train_ds = Subset(full_ds, train_idx)
    val_ds   = Subset(full_ds, val_idx)

    # Cambiar transformaciones
    train_ds.dataset.transform = train_tf
    val_ds.dataset.transform   = val_tf

    # DataLoaders
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=(DEVICE == "cuda"))
    val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, pin_memory=(DEVICE == "cuda"))

    # ======================================================
    # 🧠 Modelo: MobileNetV3 preentrenada
    # ======================================================
    model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
    model.classifier[3] = nn.Linear(1024, 2)  # salida binaria
    model = model.to(DEVICE)

    criterion = nn.CrossEntropyLoss(weight=torch.tensor([2.0, 1.0]).to(DEVICE))
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=0.5, patience=2)

    # ======================================================
    # 🚀 Función auxiliar de entrenamiento
    # ======================================================
    def run_epoch(dataloader, train: bool):
        model.train() if train else model.eval()
        total, correct, loss_sum = 0, 0, 0.0
        scaler = amp.GradScaler(device="cuda", enabled=(DEVICE == "cuda"))

        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            if train:
                optimizer.zero_grad(set_to_none=True)

            with torch.set_grad_enabled(train):
                with amp.autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
                    logits = model(x)
                    loss = criterion(logits, y)

                if train:
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

            loss_sum += loss.item() * x.size(0)
            preds = logits.argmax(1)
            correct += (preds == y).sum().item()
            total += x.size(0)

        return loss_sum / total, correct / total

    # ======================================================
    # 🔄 Loop de entrenamiento por fold
    # ======================================================
    best_acc = 0.0
    best_state = None
    epochs_no_improve = 0

    for epoch in range(1, EPOCHS + 1):
        train_loss, train_acc = run_epoch(train_dl, train=True)
        val_loss, val_acc     = run_epoch(val_dl, train=False)

        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | "
              f"train_loss={train_loss:.4f} acc={train_acc:.3f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.3f} | "
              f"lr={optimizer.param_groups[0]['lr']:.2e}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"⏹️ Early stopping: sin mejora en {PATIENCE} épocas.")
                break

    # Guardar mejor modelo del fold
    save_path = os.path.join(SAVE_DIR, f"fold{fold}_best.pt")
    torch.save(best_state, save_path)
    print(f"💾 Modelo guardado: {save_path}")

    # ======================================================
    # 📏 Evaluar métricas del fold
    # ======================================================
    model.load_state_dict(best_state)
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            preds = logits.argmax(1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    prec = precision_score(y_true, y_pred, average="macro")
    rec  = recall_score(y_true, y_pred, average="macro")
    f1   = f1_score(y_true, y_pred, average="macro")

    fold_metrics["accuracy"].append(best_acc)
    fold_metrics["precision"].append(prec)
    fold_metrics["recall"].append(rec)
    fold_metrics["f1"].append(f1)

    print(f"✅ Fold {fold} completado.")
    print(f"   Precision={prec:.3f} | Recall={rec:.3f} | F1={f1:.3f} | Acc={best_acc:.3f}")

    # ⭐ Guardar el mejor modelo global
    if best_acc > best_global_acc:
        best_global_acc = best_acc
        best_global_state = best_state
        torch.save(best_global_state, os.path.join(SAVE_DIR, "mobilenetv3_best_global.pt"))
        print(f"🏆 Nuevo mejor modelo global guardado (Acc={best_acc:.3f})")

# ======================================================
# 📊 Resultados finales promediados
# ======================================================
print("\n=== RESULTADOS FINALES (PROMEDIO CROSS-VALIDATION) ===")
for m in fold_metrics:
    mean = np.mean(fold_metrics[m])
    std  = np.std(fold_metrics[m])
    print(f"{m.capitalize():10s}: {mean:.3f} ± {std:.3f}")

print(f"\n🏁 Mejor modelo global guardado en: {os.path.join(SAVE_DIR, 'mobilenetv3_best_global.pt')}")


Entrenando en dispositivo: CUDA

📘 === FOLD 1/5 ===
Epoch 01 | train_loss=0.1395 acc=0.923 | val_loss=1.8120 acc=0.704 | lr=1.00e-03
Epoch 02 | train_loss=0.0498 acc=0.977 | val_loss=5.8757 acc=0.397 | lr=1.00e-03
Epoch 03 | train_loss=0.0385 acc=0.983 | val_loss=4.9886 acc=0.434 | lr=1.00e-03
Epoch 04 | train_loss=0.0176 acc=0.993 | val_loss=3.9773 acc=0.494 | lr=5.00e-04
Epoch 05 | train_loss=0.0129 acc=0.996 | val_loss=2.5417 acc=0.537 | lr=5.00e-04
Epoch 06 | train_loss=0.0085 acc=0.996 | val_loss=1.3748 acc=0.675 | lr=5.00e-04
Epoch 07 | train_loss=0.0052 acc=0.999 | val_loss=1.1309 acc=0.693 | lr=5.00e-04
Epoch 08 | train_loss=0.0039 acc=0.999 | val_loss=0.1408 acc=0.966 | lr=5.00e-04
Epoch 09 | train_loss=0.0035 acc=0.999 | val_loss=0.0269 acc=0.997 | lr=5.00e-04
Epoch 10 | train_loss=0.0046 acc=0.998 | val_loss=0.0923 acc=0.977 | lr=5.00e-04
Epoch 11 | train_loss=0.0194 acc=0.993 | val_loss=0.0619 acc=0.977 | lr=5.00e-04
Epoch 12 | train_loss=0.0115 acc=0.995 | val_loss=0.0727 

In [2]:
# ======================================================
# 🧪 EVALUACIÓN FINAL SOBRE EL SET DE PRUEBA
# ======================================================
import torch
import numpy as np
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

print("\n🔍 Evaluando el modelo final en el conjunto de PRUEBA...")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Transformación del set de prueba ---
test_tf = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Dataset de prueba (nunca usado en K-Fold) ---
test_ds = datasets.ImageFolder("datasets_test", transform=test_tf)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

# --- Cargar el modelo final o el mejor fold ---
model = models.mobilenet_v3_small(weights=None)
model.classifier[3] = nn.Linear(1024, 2)
model.load_state_dict(torch.load("runs_kfold/mobilenetv3_best_global.pt", map_location=DEVICE))
model.to(DEVICE)
model.eval()  

# --- Evaluación ---
y_true, y_pred, vacio_probs = [], [], []

with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        probs = F.softmax(logits, dim=1)
        preds = probs.argmax(1)

        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        vacio_probs.extend(probs[:, 1].cpu().numpy())  # prob de 'vacio'

# --- Calcular métricas ---
prec = precision_score(y_true, y_pred, average="macro")
rec  = recall_score(y_true, y_pred, average="macro")
f1   = f1_score(y_true, y_pred, average="macro")
acc  = np.mean(np.array(y_true) == np.array(y_pred))
cm   = confusion_matrix(y_true, y_pred)

# --- Mostrar resultados ---
print("\n=== RESULTADOS FINALES EN TEST ===")
print(f"Accuracy : {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1-score : {f1:.3f}")

print("\n=== MATRIZ DE CONFUSIÓN ===")
print(cm)

print("\n=== REPORTE COMPLETO ===")
clases = test_ds.classes  
print(classification_report(y_true, y_pred, target_names=clases, digits=3))



🔍 Evaluando el modelo final en el conjunto de PRUEBA...

=== RESULTADOS FINALES EN TEST ===
Accuracy : 0.995
Precision: 0.994
Recall   : 0.995
F1-score : 0.995

=== MATRIZ DE CONFUSIÓN ===
[[108   1]
 [  0  81]]

=== REPORTE COMPLETO ===
              precision    recall  f1-score   support

     Con_pez      1.000     0.991     0.995       109
       Vacio      0.988     1.000     0.994        81

    accuracy                          0.995       190
   macro avg      0.994     0.995     0.995       190
weighted avg      0.995     0.995     0.995       190

